In [2]:
!pip install sentence_transformers
!pip install wikipedia
# !wget -c "https://figshare.com/ndownloader/files/13774556"

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 85 kB 2.7 MB/s 
     |████████████████████████████████| 4.7 MB 40.4 MB/s 
     |████████████████████████████████| 1.3 MB 8.0 MB/s 
     |████████████████████████████████| 101 kB 6.0 MB/s 
     |████████████████████████████████| 596 kB 48.8 MB/s 
     |████████████████████████████████| 6.6 MB 34.2 MB/s 
  Created wheel for sentence-transformers: filename=sentence_transformers-2.2.2-py3-none-any.whl size=125938 sha256=2c6ed6ba9964807f716f3cfcefe15c9c4239330030f9c8f36dc9940a05a5555a
  Stored in directory: /root/.cache/pip/wheels/bf/06/fb/d59c1e5bd1dac7f6cf61ec0036cc3a10ab8fecaa6b2c3d3ee9
Successfully built sentence-transformers
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wh

In [3]:
import spacy
from spacy import displacy
from collections import Counter
import en_core_web_sm
from pprint import pprint
from nltk.util import ngrams
import wikipedia as wiki
import gensim
import re
import nltk
from nltk import tokenize
from sentence_transformers import SentenceTransformer
import numpy as np
import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import TfidfVectorizer


model = SentenceTransformer('bert-base-nli-mean-tokens')
nlp = en_core_web_sm.load()
nltk.download('punkt')

Downloading:   0%|          | 0.00/391 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/190 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/3.95k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/2.00 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/625 [00:00<?, ?B/s]

...

Downloading:   0%|          | 0.00/399 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/232k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/229 [00:00<?, ?B/s]

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

# Use n-grams for search

In [4]:
def keywords(sentence):

    doc = nlp(sentence)

    results = ([(X.text, X.label_)[0] for X in doc.ents])
    n_gram=[]

    for i in range(2, 5):
        n_gram.extend(ngrams(sentence.split(), i))

    results.extend([' '.join(i) for i in n_gram])
    return list(set(results))

In [5]:
def clean_wiki_text(text):
    text = re.sub(r'==.+==', '.', text)
    text = re.sub(r'\n', ' ', text)
    text = re.sub(r'\t', ' ', text)
    text = re.sub(r'\[[0-9]+\]', ' ', text)
    text = re.sub(r' +', ' ', text)
    text = re.sub(r'\. \.', '.', text)
    return text

In [6]:
def content(claim, results):
    sentences = []
    found=[]
    for i in results:
        try :
            current_page = wiki.page(i)
            if current_page not in found:
                found.append(current_page)
        except :
            continue
    titles=[i.title for i in found]
    titles=[i[0] for i in topNSimilar(claim, titles)]
    for i in found:
        if i.title not in titles:
            found.remove(i)

    for i in found:
        current_content = i.content
        sentences.extend(tokenize.sent_tokenize(clean_wiki_text(current_content)))
    return sentences

In [7]:
def topNSimilar(claim, sentList, n=5):
    distList=[]
    for i in sentList:
        document_term_matrix = TfidfVectorizer().fit_transform([i, claim])
        dist = document_term_matrix * document_term_matrix.transpose()
        distance = dist.toarray()[0][1]

        if len(distList)<=n:
            distList.append([i, distance])
        else:
            distList=sorted(distList, key=lambda x: x[1], reverse=True)
            if distance>distList[-1][1]:
                distList.pop()
                distList.append([i, distance])
    return sorted(distList, key=lambda x: x[1], reverse=True)

In [ ]:
def topNBert(claim, res):
    topNFacts=[i[0] for i in res]
    topNScore=[cosine_similarity( [model.encode(claim)], [model.encode(i)] )[0][0] for i in topNFacts]
    topN=zip(topNFacts, topNScore)
    return [list(i) for i in list(topN)]

In [8]:
claim = 'Donald Trump is the president of the united states'

In [9]:
final_results = topNBert(claim, topNSimilar(claim, content(claim, keywords(claim)), 5))

/usr/local/lib/python3.7/dist-packages/wikipedia/wikipedia.py:389: UserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 389 of the file /usr/local/lib/python3.7/dist-packages/wikipedia/wikipedia.py. To get rid of this warning, pass the additional argument 'features="lxml"' to the BeautifulSoup constructor.

  lis = BeautifulSoup(html).find_all('li')
